In [1]:
from pathlib import Path # to interact with file system.

import numpy as np # for working with arrays.
import pandas as pd # for working with data frames (tables).

from sklearn.model_selection import train_test_split # for data partition.
from sklearn.metrics import r2_score # to identify r_squared for regression model.
from sklearn.linear_model import LinearRegression # for linear regression model.
from sklearn.metrics import mean_squared_error

%matplotlib inline 
import matplotlib.pylab as plt

from dmba import regressionSummary, exhaustive_search
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import adjusted_r2_score, AIC_score, BIC_score


import os
import calendar
#import scatter_matrix, parallel_coordinates
import seaborn as sns
from sklearn import preprocessing



In [2]:
pip install dmba

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [4]:
import seaborn as sns
import matplotlib.pylab as plt

In [5]:
fact_taxi_df = pd.read_csv("fact_taxi_transport.csv", low_memory=False)
dim_Agr_df = pd.read_csv("Dim_AGR.csv")
dim_income_df = pd.read_csv("Dim_Income.csv")
dim_vehicles_df = pd.read_csv("Dim_Vehicles.csv")
dim_weather_df = pd.read_csv("New_Weather_Data.csv")
agg_dim_carpooled_df = pd.read_csv("AGG_Dim_carpooled.csv")
agg_dim_publictransport_df = pd.read_csv("AGG_Dim_PublicTransport.csv")
agg_dim_drovealone_df = pd.read_csv("AGG_Dim_DriveAlone.csv")
time_data_PU_df = pd.read_csv("time_data.csv")
time_data_DO_df = pd.read_csv("time_data.csv")
dim_location_PU_df = pd.read_csv("Dim_location.csv")
dim_location_DO_df = pd.read_csv("Dim_location.csv")
business_data_df = pd.read_csv("business_data.csv")

In [6]:
fact_taxi_df.columns

Index(['Dim_pickup_time_id', 'Dim_Dropoff_time_id', 'Dim_Pickup_Location_id',
       'Dim_Dropoff_Location_id', 'Dim_Business_id', 'Dim_Weather_id',
       'Dim_carpooled_id', 'Dim_DriveAlone_id', 'Dim_PublicTransport',
       'Trip_Distance', 'Tip_amount', 'Total_amount'],
      dtype='object')

In [7]:
business_data_df = business_data_df.rename(columns= {'Business_id': 'Dim_Business_id'})
dim_location_PU_df = dim_location_PU_df.rename(columns= {'LocationID' : 'Dim_Pickup_Location_id', 'Borough' : 'PUBorough', 'Zone' : 'PUZone'})
dim_location_DO_df = dim_location_DO_df.rename(columns= {'LocationID' : 'Dim_Dropoff_Location_id', 'Borough' : 'DOBorough', 'Zone' : 'DOZone'})
time_data_PU_df = time_data_PU_df.rename(columns= {'Timestamp' : 'PUTimestamp', 'Time_id' : 'Dim_pickup_time_id'})
time_data_DO_df = time_data_DO_df.rename(columns= {'Timestamp' : 'DOTimestamp', 'Time_id' : 'Dim_Dropoff_time_id'})
dim_weather_df = dim_weather_df.rename(columns= {'Weather_id' : 'Dim_Weather_id' })
dim_income_df = dim_income_df.rename(columns= {'LocationID' : 'Dim_Pickup_Location_id'})
dim_income_df = dim_income_df.rename(columns= {'LocationID' : 'Dim_Pickup_Location_id'})

In [8]:
taxi_df = business_data_df[['Industry', 'Dim_Business_id']].merge(fact_taxi_df[['Dim_pickup_time_id', 'Dim_Dropoff_time_id', 'Dim_Pickup_Location_id','Dim_Dropoff_Location_id', 'Dim_Business_id', 'Dim_Weather_id','Dim_carpooled_id', 'Dim_DriveAlone_id', 'Dim_PublicTransport','Trip_Distance', 'Tip_amount', 'Total_amount']], on = 'Dim_Business_id', how = 'right')
taxi_df = dim_location_PU_df[['Dim_Pickup_Location_id', 'PUBorough', 'PUZone']].merge(taxi_df[['Industry', 'Dim_Business_id', 'Dim_pickup_time_id','Dim_Dropoff_time_id', 'Dim_Pickup_Location_id','Dim_Dropoff_Location_id', 'Dim_Weather_id', 'Dim_carpooled_id','Dim_DriveAlone_id', 'Dim_PublicTransport', 'Trip_Distance','Tip_amount', 'Total_amount']], on = 'Dim_Pickup_Location_id', how = 'right')
taxi_df = dim_location_DO_df[['Dim_Dropoff_Location_id', 'DOBorough', 'DOZone']].merge(taxi_df[['Industry', 'Dim_Business_id', 'Dim_pickup_time_id', 'PUBorough', 'PUZone','Dim_Dropoff_time_id', 'Dim_Pickup_Location_id','Dim_Dropoff_Location_id', 'Dim_Weather_id', 'Dim_carpooled_id','Dim_DriveAlone_id', 'Dim_PublicTransport', 'Trip_Distance','Tip_amount', 'Total_amount']], on = 'Dim_Dropoff_Location_id', how = 'right')
taxi_df = time_data_PU_df[['PUTimestamp', 'Dim_pickup_time_id']].merge(taxi_df[['Dim_Dropoff_Location_id', 'DOBorough', 'DOZone', 'Industry','Dim_Business_id', 'Dim_pickup_time_id', 'PUBorough', 'PUZone','Dim_Dropoff_time_id', 'Dim_Pickup_Location_id', 'Dim_Weather_id','Dim_carpooled_id', 'Dim_DriveAlone_id', 'Dim_PublicTransport','Trip_Distance', 'Tip_amount', 'Total_amount']], on = 'Dim_pickup_time_id', how = 'right')
taxi_df = time_data_DO_df[['DOTimestamp', 'Dim_Dropoff_time_id']].merge(taxi_df[['Dim_Dropoff_Location_id', 'DOBorough', 'DOZone', 'Industry','Dim_Business_id', 'Dim_pickup_time_id','PUTimestamp', 'PUBorough', 'PUZone','Dim_Dropoff_time_id', 'Dim_Pickup_Location_id', 'Dim_Weather_id','Dim_carpooled_id', 'Dim_DriveAlone_id', 'Dim_PublicTransport','Trip_Distance', 'Tip_amount', 'Total_amount']], on = 'Dim_Dropoff_time_id', how = 'right')
taxi_df = dim_weather_df[['Dim_Weather_id', 'city_name', 'weather_type', 'Temperature', 'Date_Time']].merge(taxi_df[['Dim_Dropoff_Location_id', 'DOBorough', 'DOZone', 'Industry','Dim_Business_id', 'Dim_pickup_time_id','PUTimestamp', 'PUBorough', 'PUZone','Dim_Dropoff_time_id','DOTimestamp', 'Dim_Pickup_Location_id', 'Dim_Weather_id','Dim_carpooled_id', 'Dim_DriveAlone_id', 'Dim_PublicTransport','Trip_Distance', 'Tip_amount', 'Total_amount']], on = 'Dim_Weather_id', how = 'right')


In [14]:
taxi_df.head(5)

,Dim_Weather_id,city_name,weather_type,Temperature,Date_Time,Dim_Dropoff_Location_id,DOBorough,DOZone,Industry,Dim_Business_id,...,PUZone,Dim_Dropoff_time_id,DOTimestamp,Dim_Pickup_Location_id,Dim_carpooled_id,Dim_DriveAlone_id,Dim_PublicTransport,Trip_Distance,Tip_amount,Total_amount
0,W5,Bronx,sky is clear,257-290,4/3/2023 11:00,119,Bronx,Highbridge,Electronic & Appliance Service,B1,...,Inwood,T30,01-03-2023 00:29,127,C108,P109,D110,7.4,0.0,27.8
1,W5,Bronx,sky is clear,257-290,4/3/2023 12:00,119,Bronx,Highbridge,Electronic & Appliance Service,B1,...,Inwood,T30,01-03-2023 00:29,127,C108,P109,D110,7.4,0.0,27.8
2,W5,Bronx,sky is clear,257-290,4/3/2023 13:00,119,Bronx,Highbridge,Electronic & Appliance Service,B1,...,Inwood,T30,01-03-2023 00:29,127,C108,P109,D110,7.4,0.0,27.8
3,W5,Bronx,sky is clear,257-290,4/3/2023 14:00,119,Bronx,Highbridge,Electronic & Appliance Service,B1,...,Inwood,T30,01-03-2023 00:29,127,C108,P109,D110,7.4,0.0,27.8
4,W5,Bronx,sky is clear,257-290,9/3/2023 5:00,119,Bronx,Highbridge,Electronic & Appliance Service,B1,...,Inwood,T30,01-03-2023 00:29,127,C108,P109,D110,7.4,0.0,27.8


In [11]:
dim_vehicles_df.columns

Index(['LocationID', 'Geographic Area Name', 'Geographic Area Name - County',
       'Zone', 'No vehicle available', '1 vehicle available',
       '2 vehicles available', '3 or more vehicles available',
       'Transportation_type', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12'],
      dtype='object')

In [12]:
dim_income_df.columns

Index(['Geographic Area Name', 'Geographic Area Name - County', 'Zone',
       'LocationID', 'Earnings $1 to $9,999 or loss',
       'Earnings $10,000 to $14,999', 'Earnings $15,000 to $24,999',
       'Earnings $25,000 to $34,999', 'Earnings!!$35,000 to $49,999',
       'Earnings $50,000 to $64,999', 'earnings $65,000 to $74,999',
       'earnings $75,000 or more', 'Transportation_type'],
      dtype='object')

In [15]:
agg_dim_carpooled_df.columns

Index(['GG', 'Zones', 'Transportation_Type', '3 or more vehicles available',
       '2 vehicles available', '1 vehicle available', 'No vehicle available',
       'Male', 'Female', '16 to 19 years', '20 to 24 years', '25 to 44 years',
       '45 to 54 years', '55 to 59 years', '60 years and over',
       'Earnings $1 to $9,999 or loss', 'Earnings $10,000 to $14,999',
       'Earnings $15,000 to $24,999', 'Earnings $25,000 to $34,999',
       'Earnings $35,000 to $49,999', 'Earnings $50,000 to $64,999',
       'earnings $65,000 to $74,999', 'earnings $75,000 or more',
       'Foreign born - Not in US born',
       'Foreign born - Naturalized U.S. citizen', 'Foreign born'],
      dtype='object')